In [0]:
messy_data_file_path = "/Volumes/workspace/default/data_clean_python/marketing_campaign_data_messy.csv"

In [0]:
%python
import pandas as pd
import numpy as np

#Load Messy Data

df = pd.read_csv(messy_data_file_path)

print(f"Loaded Dataset: {df.shape[0]} rows, {df.shape[1]} columns")

In [0]:
df

## Header Cleaning

In [0]:
print(df.columns.tolist())

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print("Fix Applied")
print(df.columns.tolist())

## Type Conversion & Currency Cleaning

In [0]:
dirty_spend_mask = df['spend'].astype(str).str.contains(r'\$')
print(df.loc[dirty_spend_mask, ['campaign_id', 'spend']].head(3))

df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]', '', regex = True)

df['spend'] = pd.to_numeric(df['spend'])

print("Fix Applied")
print(df.loc[dirty_spend_mask, ['campaign_id', 'spend']].head(3))

## Categorical Typos (Fuzzy Logic)

In [0]:
print(df['channel'].unique())

cleanup_map = {
    'Facebok' : 'Facebook',
    'Tik_Tok' : 'TikTok',
    'Gogle' : 'Google',
    'Insta_gram' : 'Instagram',
    'E-mail' : 'Email',
    'N/A' : np.nan  #Handling Ghost value
}

df['channel'] = df['channel'].replace(cleanup_map)


print("Fix Applied")
print(df['channel'].unique())


## Handling Mixed Booleans

In [0]:
print(df['active'].unique())

bool_map = {
    'Y' : True,
    'No' : False,
    '0' : False,
    "1" : True,
    "Yes" : True,
    1 : True,
    0 : False, 
    'True' : True,
    'False' : False
}
df['active'] = df['active'].replace(bool_map)

print("Fix Applied")
print(df['active'].unique())

## Date Parsing

In [0]:
print(df['start_date'].dtype)

df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], dayfirst=True, errors='coerce')

print("Fix Applied")
print(df['start_date'].dtype)

## Logical Integrity (Clicks vs Impressions)

In [0]:
df = df.loc[:, ~df.columns.duplicated()]

In [0]:
df

In [0]:
impossible_mask = df['clicks'] > df['impressions']

print(df.loc[impossible_mask, ['campaign_id', 'clicks', 'impressions']].head(3))


## Logical Integrity (Time Travel)

In [0]:
time_travel_mask = df['start_date'] > df['end_date']

print(df.loc[time_travel_mask, ['campaign_id', 'start_date', 'end_date']].head(3))

df.loc[time_travel_mask, 'end_date'] = df.loc[time_travel_mask, 'start_date'] + pd.Timedelta(days=30)

print("Fix Applied")

print(df.loc[time_travel_mask, ['campaign_id', 'start_date', 'end_date']].head(3))

## Handling Outliers (Winsorizing)

In [0]:
Q1 = df['spend'].quantile(0.25)
Q3 = df['spend'].quantile(0.75)

IQR = Q3 - Q1
upper_limit = Q3 + (3 * IQR)

outlier_mask = df['spend'] > upper_limit

print(df.loc[outlier_mask, ['campaign_id', 'spend']].head(3))

print("Fix Applied")

df.loc[outlier_mask, 'spend'] = upper_limit
print(df.loc[outlier_mask, ['campaign_id', 'spend']].head(3))

## String Parsing (Feature Extraction)

In [0]:
print(df['campaign_name'].head(3))

df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')

print("Fix Applied")
print(df[['campaign_name', 'season']].head(3))
